In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from graphframes import GraphFrame

import findspark
findspark.init()


# Initialize Spark Context
conf = SparkConf().setAppName("GNNRecommender")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# Define vertices (users, artists, tags)
users = spark.createDataFrame([
    (1, "User1"),
    (2, "User2")
], ["id", "name"])

artists = spark.createDataFrame([
    (3, "Artist1"),
    (4, "Artist2")
], ["id", "name"])

tags = spark.createDataFrame([
    (5, "Tag1"),
    (6, "Tag2")
], ["id", "name"])

# Define edges (User-Artist, Artist-Tag)
userArtistEdges = spark.createDataFrame([
    (1, 3, "listens"),
    (2, 4, "listens")
], ["src", "dst", "relationship"])

artistTagEdges = spark.createDataFrame([
    (3, 5, "tagged"),
    (4, 6, "tagged")
], ["src", "dst", "relationship"])

# Combine all vertices and edges
vertices = users.union(artists).union(tags)
edges = userArtistEdges.union(artistTagEdges)

# Create the graph
graph = GraphFrame(vertices, edges)

# Show vertices and edges
graph.vertices.show()
graph.edges.show()

# Stop the SparkContext
sc.stop()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Cannot assign requested address: bind: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:565)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.netBind(ServerSocketChannelImpl.java:344)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:301)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChan

In [6]:
sc.stop()

NameError: name 'sc' is not defined

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("PySpark Installation Test").getOrCreate()
df = spark.createDataFrame([(1, "Hello"), (2, "World")], ["id", "message"])
df.show()

Py4JJavaError: An error occurred while calling o43.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (172.28.145.115 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.